Se transforman los datos desde Stata a CSV

In [1]:
import pandas as pd

# Leer sin convertir categorías
df = pd.read_stata("casen_en_Pandemia_2020_STATA.dta", convert_categoricals=False)

reader = pd.io.stata.StataReader("casen_en_Pandemia_2020_STATA.dta")
value_labels = reader.value_labels()
reader.close()

# Guardar
df.to_csv("CASEN_PANDEMIA_2020.csv", index=False, encoding='utf-8-sig')
print(f"{len(df)} filas guardadas")

C:\Users\Liz\AppData\Local\Temp\ipykernel_12292\835817429.py:4: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  df = pd.read_stata("casen_en_Pandemia_2020_STATA.dta", convert_categoricals=False)
C:\Users\Liz\AppData\Local\Temp\ipykernel_12292\835817429.py:8: FutureWarning: The StataReader.close() method is not part of the public API and will be removed in a future version without notice. Using StataReader as a context manager is the only supported method.
  reader.close()


185437 filas guardadas


Inspeccionar los datos del cansen para evaluarlos

In [8]:
df.head()



,folio,o,id_persona,id_vivienda,region,provincia,comuna,zona,area,segmento,...,v20,v20_esp,v20_red,v22,v23,v23_sistema,v23_cajon,v27,v28,v29
0,1.101100e+11,1,5,1.101100e+09,1,11,1101,1,1,11011001,...,1,,2.0,1.0,2,NaN,NaN,1,NaN,NaN
1,1.101100e+11,2,6,1.101100e+09,1,11,1101,1,1,11011001,...,1,,2.0,1.0,2,NaN,NaN,1,NaN,NaN
2,1.101100e+11,1,32,1.101100e+09,1,11,1101,1,1,11011001,...,1,,2.0,1.0,1,1.0,NaN,1,NaN,NaN
3,1.101100e+11,2,31,1.101100e+09,1,11,1101,1,1,11011001,...,1,,2.0,1.0,1,1.0,NaN,1,NaN,NaN
4,1.101100e+11,3,30,1.101100e+09,1,11,1101,1,1,11011001,...,1,,2.0,1.0,1,1.0,NaN,1,NaN,NaN


Ver datos de pobreza

In [9]:
print(df['pobreza'].value_counts())

pobreza
3.0    164042
2.0     12862
1.0      8435
Name: count, dtype: int64


In [10]:
print('pobreza' in df.columns)

True


In [11]:
print(df['pobreza_sinte'].value_counts())

pobreza_sinte
3    158565
2     15292
1     11580
Name: count, dtype: int64


In [17]:
print('nse' in df.columns)

False


In [18]:
import numpy as np
conditions = [
    df['pobreza'] == 1,  # Indigentes
    df['pobreza'] == 2,  # Pobres
    df['ypchautcor'] < df['ypchautcor'].quantile(0.75),  # Medio
    df['ypchautcor'] >= df['ypchautcor'].quantile(0.75)  # Alto
]

choices = ['E', 'D', 'C2-C1', 'AB']

df['nse_aproximado'] = np.select(conditions, choices, default='C2-C1')

C:\Users\Liz\AppData\Local\Temp\ipykernel_12292\4292242526.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['nse_aproximado'] = np.select(conditions, choices, default='C2-C1')


Ya investigado la documentacion entregada por la web de la base de datos y el dataframe, tenemos el nombre de las variables para crear un dataframe de pobreza con CASEN EN PANDEMIA 2020

In [36]:
# Variables correctas disponibles
variables_necesarias = ['folio', 'id_persona','provincia','comuna','pobreza', 'lp','li','pobreza_sinte', 'nse_aproximado',
                          'edad', 'sexo', 'ytotcorh', 'yoprcorh','expr']

df_pobreza = df[variables_necesarias].copy()

# Eliminar filas donde pobreza sea nula
df_pobreza = df_pobreza.dropna(subset=['pobreza'])

# Ver distribución de pobreza
print(" DISTRIBUCIÓN DE POBREZA POR INGRESOS ")
print(df_pobreza['pobreza'].value_counts().sort_index())

print("\n DISTRIBUCIÓN DE Pobreza por Sistema de Imputación de Ingresos No Trabajo y Empleo")
print(df_pobreza['pobreza_sinte'].value_counts().sort_index())

print("\n DISTRIBUCIÓN DE NIVEL SOCIOECONÓMICO (NSE) ")
print(df_pobreza['nse_aproximado'].value_counts().sort_index())

print(f"\nTotal personas con datos: {len(df_pobreza)}")

 DISTRIBUCIÓN DE POBREZA POR INGRESOS 
pobreza
1.0      8435
2.0     12862
3.0    164042
Name: count, dtype: int64

 DISTRIBUCIÓN DE Pobreza por Sistema de Imputación de Ingresos No Trabajo y Empleo
pobreza_sinte
1     11580
2     15292
3    158467
Name: count, dtype: int64

 DISTRIBUCIÓN DE NIVEL SOCIOECONÓMICO (NSE) 
nse_aproximado
AB        46270
C2-C1    117772
D         12862
E          8435
Name: count, dtype: int64

Total personas con datos: 185339


In [37]:
df_pobreza.info()

<class 'pandas.core.frame.DataFrame'>
Index: 185339 entries, 0 to 185436
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   folio           185339 non-null  float64
 1   id_persona      185339 non-null  int16  
 2   provincia       185339 non-null  int16  
 3   comuna          185339 non-null  int16  
 4   pobreza         185339 non-null  float64
 5   lp              185339 non-null  int32  
 6   li              185339 non-null  int32  
 7   pobreza_sinte   185339 non-null  int8   
 8   nse_aproximado  185339 non-null  object 
 9   edad            185339 non-null  int16  
 10  sexo            185339 non-null  int8   
 11  ytotcorh        185339 non-null  float64
 12  yoprcorh        185339 non-null  float32
 13  expr            185339 non-null  int16  
dtypes: float32(1), float64(3), int16(5), int32(2), int8(2), object(1)
memory usage: 11.3+ MB


In [38]:
df_pobreza.head()

,folio,id_persona,provincia,comuna,pobreza,lp,li,pobreza_sinte,nse_aproximado,edad,sexo,ytotcorh,yoprcorh,expr
0,1.101100e+11,5,11,1101,3.0,174131,116087,3,C2-C1,34,2,390833.0,150000.0,67
1,1.101100e+11,6,11,1101,3.0,174131,116087,3,C2-C1,4,2,390833.0,150000.0,67
2,1.101100e+11,32,11,1101,3.0,174131,116087,3,C2-C1,45,1,947583.0,372000.0,67
3,1.101100e+11,31,11,1101,3.0,174131,116087,3,C2-C1,5,2,947583.0,372000.0,67
4,1.101100e+11,30,11,1101,3.0,174131,116087,3,C2-C1,19,2,947583.0,372000.0,67


In [39]:
df_pobreza.to_csv("CASEN_PANDEMIA_POBREZA_GEOGRAFICA_2020.csv", index=False, encoding='utf-8-sig')